# Machine Translation, human editors research: Retrieve CX Sample

26 January 2021

[Task](https://phabricator.wikimedia.org/T290906)

The following process will be used to pull a sample that meets sample specifications, detailed in the [task description](https://phabricator.wikimedia.org/T290906)

1. Write and run a query to generate a random sample of translation_ids that meet the requested article specifications using the CX databases (with joins to other data sources as needed).
2. Use the results of the query to populate a spreadsheet that contains the translation_ids and requested meta data for each article in the sample.
3. Use the translation_ids to obtain side by side comparison of the source, MT output and CX publication for each article (Provided in either spreadsheet form or viewed within the debugger tool interface)


Notes:
- Not enough cx translated articles to meet all the specified requirements requirements. Please see comments in task to learn the details of the adjustments. https://phabricator.wikimedia.org/T290906
- Topic Model comes from research model of https://github.com/geohci/wikipedia-language-agnostic-topic-classification, which uses the ORES model.
- Only published translations reviewed.
- In the cx_translations translation_progress field, the human and machine translation percentages do not equal 100 and there are some instances of the human percent modified exceeding 100. Need to investigate causes and sources of this. Per the published translation data [page](https://www.mediawiki.org/wiki/Content_translation/Published_translations?useskin=vector-2022), "Any edits to machine translation output are considered as manual edits. The percentages are calculated at section level. any indicates the total translation (any=human+mt). Content Translation does not demand full translation of the source article. Users can freely translate as many or as few sections as they want.
- CX algorithm score = unmodified MT content. Per Santosh " The overal MT percentage is what we have in database and that is what we use for deciding whether the translation can be published or not." See furhter information on CX score algorithm at https://github.com/wikimedia/mediawiki-extensions-ContentTranslation/blob/master/doc/MTAbuseCalculation.md


In [4]:
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from datetime import date

from wmfdata import hive, mariadb

You are using wmfdata v1.3.2, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


# Pull Translation IDs and Meta Data

In [477]:
# pull translation ids and needed meta data

query = """

SELECT
    translation_id,
    translation_target_title,
    translation_started_by AS translator,
    translation_source_language,
    translation_target_language,
    translation_start_timestamp,
    cc.cxc_origin AS mt_service,
    translation_last_updated_timestamp,
    json_extract(translation_progress, '$.mt') AS mt_translated_percent, 
    CASE
    WHEN json_extract(translation_progress, '$.human') < 0.10 THEN 'less than 10%'
    WHEN (json_extract(translation_progress, '$.human') > 0.10 AND json_extract(translation_progress, '$.human') < 0.51) THEN 'between 11 and 50%'
    ELSE 'over 50 percent'
    END AS 'percent_MT_modified'-- need to parse a JSON object
FROM
    cx_translations
    JOIN 
    cx_corpora AS cc
    ON translation_id = cc.cxc_translation_id
WHERE
    translation_target_language IN ('sq', 'id', 'zh')
    AND translation_source_language = 'en'
    AND translation_status = 'published'
"""

In [478]:
translations = mariadb.run(commands = query, dbs = 'wikishared', use_x1 = True, format="pandas")

In [479]:
translations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34719 entries, 0 to 34718
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   translation_id                      34719 non-null  int64 
 1   translation_target_title            34719 non-null  object
 2   translator                          34719 non-null  int64 
 3   translation_source_language         34719 non-null  object
 4   translation_target_language         34719 non-null  object
 5   translation_start_timestamp         34719 non-null  object
 6   translation_last_updated_timestamp  34719 non-null  object
 7   mt_translated_percent               34719 non-null  object
 8   percent_MT_modified                 34719 non-null  object
dtypes: int64(2), object(7)
memory usage: 2.4+ MB


# Find Translator Registration Date  

This will be used to meet the following article specification: "-50% of the articles should have been published by a ‘newer’ editor, defined here as an account created no longer than 2 years prior." To obtain this data, we'll use the https://www.mediawiki.org/wiki/Extension:CentralAuth/globaluser_table.

In [480]:
# obtain users global registration date
#FIXME ADD way to just go through translator list
query = '''
SELECT
    globaluser.gu_id AS user_id,
    globaluser.gu_registration AS registration_date
FROM
    globaluser
'''

In [481]:
registrations = mariadb.run(commands = query, dbs = 'centralauth', format="pandas")

In [482]:
# Join with cx_translators data
cx_translations_wexp = translations.join(registrations.set_index('user_id'), on = 'translator', how = 'left')

In [484]:
cx_translations_wexp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34719 entries, 0 to 34718
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   translation_id                      34719 non-null  int64 
 1   translation_target_title            34719 non-null  object
 2   translator                          34719 non-null  int64 
 3   translation_source_language         34719 non-null  object
 4   translation_target_language         34719 non-null  object
 5   translation_start_timestamp         34719 non-null  object
 6   translation_last_updated_timestamp  34719 non-null  object
 7   mt_translated_percent               34719 non-null  object
 8   percent_MT_modified                 34719 non-null  object
 9   registration_date                   34719 non-null  object
dtypes: int64(2), object(8)
memory usage: 2.6+ MB


In [485]:
# convert to datetime
cx_translations_wexp.registration_date = pd.to_datetime(cx_translations_wexp['registration_date'], format='%Y%m%d%H%M%S')
cx_translations_wexp.translation_start_timestamp = pd.to_datetime(cx_translations_wexp['translation_start_timestamp'], format='%Y%m%d%H%M%S')

In [486]:
# find the time difference between translator registration date and translation
cx_translations_wexp['time_since_registration'] = cx_translations_wexp.translation_start_timestamp - cx_translations_wexp.registration_date
cx_translations_wexp['time_since_registration'] = cx_translations_wexp.time_since_registration/ np.timedelta64(1, "Y")

# Find Articles that belong to Biography or Earth and Environment Topics

In [487]:
#Find all Page IDS and titles Associated with Relevant Topics using topics table table
## Reference: https://github.com/conniecc1/update_content_pv/blob/main/update_content_pv.ipynb
##  https://github.com/jenniferwang-wmf/content_edit_daily/blob/master/content_edit_daily.ipynb
query = """
SELECT
    mwh.wiki_db,
    mwh.page_id,
    mwh.page_title,
    topics.topic,
    Count(*) As edit_count
FROM wmf.mediawiki_history AS mwh
INNER JOIN canonical_data.wikis cd ON mwh.wiki_db = cd.database_code AND
    cd.database_group = "wikipedia"
INNER JOIN isaacj.article_topics_outlinks_2021_11 topics ON (
  cd.database_code =  topics.wiki_db
  AND mwh.page_id = topics.pageid
  AND topics.score >= 0.5
  )
WHERE 
    mwh.event_entity = 'revision'
    AND mwh.event_type = 'create' 
    AND mwh.page_namespace = 0
    AND mwh.snapshot = '{SNAPSHOT}'
    AND mwh.wiki_db IN ('sqwiki', 'idwiki', 'zhwiki') 
    AND(topics.topic LIKE 'Culture.Biography.Biography%' OR
    topic LIKE 'STEM.Earth_and_environment%')
GROUP BY
   mwh.wiki_db,
    mwh.page_id,
    mwh.page_title,
    topics.topic
"""

In [488]:
pages = hive.run(query.format(SNAPSHOT='2021-12'))

In [503]:
# join with table to add topic info.
# inner join to exclude all articles that don't fit into these 2 categories
cx_translations_wexp_andtopic = cx_translations_wexp.merge(pages, how = 'inner', left_on = 'translation_target_title', right_on = 'page_title')

In [504]:
cx_translations_wexp_andtopic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3511 entries, 0 to 3510
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   translation_id                      3511 non-null   int64         
 1   translation_target_title            3511 non-null   object        
 2   translator                          3511 non-null   int64         
 3   translation_source_language         3511 non-null   object        
 4   translation_target_language         3511 non-null   object        
 5   translation_start_timestamp         3511 non-null   datetime64[ns]
 6   translation_last_updated_timestamp  3511 non-null   object        
 7   mt_translated_percent               3511 non-null   object        
 8   percent_MT_modified                 3511 non-null   object        
 9   registration_date                   3511 non-null   datetime64[ns]
 10  time_since_registration 

# Identify Experienced Editors 

The other half of articles should have been published by editors with CX publications beginning at least 3 years prior. To calculate this, we need to group by all users (translation_started_by) and find where min date is >= 3 years max date.

In [506]:
# Find first translation dates
first_translation_dates = cx_translations_wexp_andtopic.groupby('translator').agg({'translation_start_timestamp':['min']}).reset_index()

In [507]:
# Fix column names for merging
first_translation_dates.columns = first_translation_dates.columns.get_level_values(0)
first_translation_dates_fixed = first_translation_dates.rename(columns={"translator": "translator", "translation_start_timestamp": "first_translation_date"})

In [508]:
# Join First Dates to translation id data frame
cx_translations_wexp_andtopic = cx_translations_wexp_andtopic.join(first_translation_dates_fixed.set_index('translator'), on = 'translator', how = 'right')

In [509]:
# Find last translation date
last_translation_dates = cx_translations_wexp_andtopic.groupby('translator').agg({'translation_start_timestamp':['max']}).reset_index()

In [510]:
# Fix column names for merging
last_translation_dates.columns = last_translation_dates.columns.get_level_values(0)
last_translation_dates_fixed = last_translation_dates.rename(columns={"translator": "translator", "translation_start_timestamp": "last_translation_date"})

In [511]:
# Join Last Dates to translation id data frame
cx_translations_wexp_andtopic = cx_translations_wexp_andtopic.join(last_translation_dates_fixed.set_index('translator'), on = 'translator', how = 'right')

In [512]:
# Find difference in first and last translation date

cx_translations_wexp_andtopic['time_since_first_translation'] = cx_translations_wexp_andtopic.last_translation_date - cx_translations_wexp_andtopic.first_translation_date

In [513]:
cx_translations_wexp_andtopic['time_since_first_translation'] = cx_translations_wexp_andtopic.time_since_first_translation/ np.timedelta64(1, "Y")

In [514]:
# add column to categorize translator experience
## 50% of the articles should have been published by a ‘newer’ editor, defined here as an account created no longer than 2 years prior. 

cx_translations_wexp_andtopic['user_tenure'] = np.select(
    [
        cx_translations_wexp_andtopic['time_since_registration'].between(0, 2, inclusive=False),
    ], 
    [
        'junior', 
    ], 
    default='senior'
)

In [515]:
# add column to categorize translator experience
## The other half of articles should have been published by editors with CX publications beginning at least 3 years prior.

cx_translations_wexp_andtopic['translator_experience'] = np.select(
    [
        cx_translations_wexp_andtopic['time_since_first_translation'].ge(3),
    ], 
    [
        'experienced', 
    ], 
    default='new'
)

# QA Checks

In [43]:
# find if there are any duplicate translation ids

ids = cx_translations_wexp['translation_id']
cx_translations_wexp[ids.isin(ids[ids.duplicated()])]

,translation_id,translator,translation_target_language,translation_start_timestamp,translation_target_revision_id,translation_last_updated_timestamp,percent modified,registration_date,diff_years,user_tenure


In [ ]:
translators = cx_translations_wexp['translator']
cx_translations_wexp[translators.isin(translators[translators.duplicated()])]

In [ ]:
Confirmed there are no duplicate translation ids but there are duplicate translators as expected

# Save samples table

In [516]:
cx_translations_wexp_andtopic.to_csv("Data/translations_raw_data.csv")

# Pull Sample

This was done using the raw data in excel.

**Requirements**
- 50 random articles for each wiki
- Need to pull 50% with newer editor and 50% older (25 and 25)
- Need to pull equal split across the three percent modified categories
- At least 10 or more individual editors, with no individual editor contributing more than 5 articles